# Constructing API URL
* Accessing the [Copernicus Data Space Ecosystem](https://dataspace.copernicus.eu/)
* Constructing URL query strings [Requests: passing parameters in URLs](https://requests.readthedocs.io/en/latest/user/quickstart/#passing-parameters-in-urls)

In [16]:
import requests
import urllib
import json
import pandas
import datetime
import pathlib
import geopandas

In [2]:
SCHEME = "https"
NETLOC_API = "catalogue.dataspace.copernicus.eu"
ODATA_API = "odata/v1/Products"
data_url = urllib.parse.urlunparse( (SCHEME, NETLOC_API, ODATA_API, "", "", "", ))

## APIs - OData
* A Catalogue API - return satellite layers in a time range of location as a data frame
* Documentation at [OData API](https://documentation.dataspace.copernicus.eu/APIs/OData.html)
* Query made up of options: filter, orderby, top, skip, count, expand
  * Can filter by [Attributes for query](https://documentation.dataspace.copernicus.eu/APIs/OData.html#list-of-odata-query-attributes-by-collection) - [SENTINEL-2 attributes](https://catalogue.dataspace.copernicus.eu/odata/v1/Attributes(SENTINEL-2)) e.g. cloud cover
  * Can filter by 'Collection/Name' e.g. ‘SENTINEL-3’, ‘SENTINEL-2’, ‘SENTINEL-1’
* Download documetnation - [compressed product download](https://documentation.dataspace.copernicus.eu/APIs/OData.html#compressed-product-download)
* Other catalogue APIs include STACK API, OpenSearch, Sentinel Hub Catalogue API
## APIs - openEO API
* A standardised API to access & process datasets in the ecosystem using intuitive programming libraries
* Auto-scalled on cloud infrastructure that hosts the data
## APIs - On-demand processing
* Serverless data computation service that triggers the generation of higher level user products from lower-level products
* Uses ESA CFIs and other pre-defined processors
* Query API to brows available workflows, submit jobs, review status, retrieve results using we browser
## APIS - Sentinel Hub
* RESTful API interface for accessing stallite imagry archives
* Access of raw staellite data, rendered images, statistical analysis

# Example query - Not Sentinel 2
* Not SENTINEL-2

In [27]:
api_parameters = {
    "$filter": r"not (Collection/Name eq 'SENTINEL-2') and ContentDate/Start gt 2022-05-03T00:00:00.000Z and ContentDate/Start lt 2022-05-03T00:10:00.000Z",
    "$orderby": "ContentDate/Start",
    "$top": 100}
response = requests.get(data_url, params=api_parameters)
response.url

'https://catalogue.dataspace.copernicus.eu/odata/v1/Products?%24filter=not+%28Collection%2FName+eq+%27SENTINEL-2%27%29+and+ContentDate%2FStart+gt+2022-05-03T00%3A00%3A00.000Z+and+ContentDate%2FStart+lt+2022-05-03T00%3A10%3A00.000Z&%24orderby=ContentDate%2FStart&%24top=100'

In [28]:
response_json = response.json()
values = pandas.DataFrame.from_dict(response_json["value"])

In [29]:
response_json['@odata.context'] # '@odata.nextLink'

'$metadata#Products'

In [30]:
values.head(2)

,@odata.mediaContentType,Id,Name,ContentType,ContentLength,OriginDate,PublicationDate,ModificationDate,Online,EvictionDate,S3Path,Checksum,ContentDate,Footprint,GeoFootprint
0,application/octet-stream,1d42f2d3-2456-485f-a93e-92f08bdd5c51,S1A_OPER_AUX_GNSSRD_POD__20220510T020122_V2022...,application/octet-stream,2663000,2022-05-10T02:30:11.130000Z,2023-10-25T13:45:19.736051Z,2023-11-14T22:50:17.707935Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-1/AUX/AUX_GNSSRD/2022/05/03/S...,"[{'Value': '6a99572d2baaa3c9a83bd851ba3ba70f',...","{'Start': '2022-05-03T00:00:04.000000Z', 'End'...",None,None
1,application/octet-stream,5c744d5c-c082-4a34-a181-81cde73cd25d,S1B_OPER_AUX_GNSSRD_POD__20220510T023113_V2022...,application/octet-stream,2664746,2022-05-10T03:00:10.749000Z,2023-10-25T13:45:38.892646Z,2023-11-14T22:59:42.253981Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-1/AUX/AUX_GNSSRD/2022/05/03/S...,"[{'Value': 'b07856cd6772060149b449ebd8352ca3',...","{'Start': '2022-05-03T00:00:08.000000Z', 'End'...",None,None


# Example Query - select Sentinel 2

In [31]:
api_parameters = {
    "$filter": r"(Collection/Name eq 'SENTINEL-2') and ContentDate/Start gt 2022-05-03T00:00:00.000Z and ContentDate/Start lt 2022-05-03T00:10:00.000Z",
    "$orderby": "ContentDate/Start",
    "$top": 100}
response = requests.get(data_url, params=api_parameters)
response.url

'https://catalogue.dataspace.copernicus.eu/odata/v1/Products?%24filter=%28Collection%2FName+eq+%27SENTINEL-2%27%29+and+ContentDate%2FStart+gt+2022-05-03T00%3A00%3A00.000Z+and+ContentDate%2FStart+lt+2022-05-03T00%3A10%3A00.000Z&%24orderby=ContentDate%2FStart&%24top=100'

In [32]:
response_json = response.json()
values = pandas.DataFrame.from_dict(response_json["value"])
values.head(2)

,@odata.mediaContentType,Id,Name,ContentType,ContentLength,OriginDate,PublicationDate,ModificationDate,Online,EvictionDate,S3Path,Checksum,ContentDate,Footprint,GeoFootprint
0,application/octet-stream,b2ac733d-efa4-49e9-9edc-fb37e2a7f938,S2B_OPER_AUX_GNSSRD_POD__20220510T031816_V2022...,application/octet-stream,2650202,2022-05-10T03:32:10.993000Z,2023-10-25T13:45:42.482692Z,2023-11-29T14:03:28.169944Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/AUX/AUX_GNSSRD/2022/05/03/S...,"[{'Value': '5d7e3d41cedc5331584cdc6b92692b6c',...","{'Start': '2022-05-03T00:00:04.000000Z', 'End'...",None,None
1,application/octet-stream,a449f701-d29a-4ce9-b838-33061ee1d573,S2A_OPER_AUX_GNSSRD_POD__20220510T030151_V2022...,application/octet-stream,2281403,2022-05-10T03:32:11.048000Z,2023-10-25T13:45:41.052411Z,2023-11-29T14:03:27.695824Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/AUX/AUX_GNSSRD/2022/05/03/S...,"[{'Value': '56a1a8b66c90102c86511dfa991babc3',...","{'Start': '2022-05-03T00:00:07.000000Z', 'End'...",None,None


## Define date range

In [54]:
start_date = datetime.datetime(year=2022, month=5, day=3)
time_range = datetime.timedelta(hours=10)
start_string = start_date.strftime("%Y-%m-%dT%X.%fZ")
end_string = (start_date + time_range).strftime("%Y-%m-%dT%X.%fZ")
data_collection = "SENTINEL-2"

In [55]:
api_parameters = {
    "$filter": f"(Collection/Name eq '{data_collection}') and ContentDate/Start gt {start_string} and ContentDate/Start lt {end_string}",
    "$orderby": "ContentDate/Start",
    "$top": 100}
response = requests.get(data_url, params=api_parameters)
response.url

'https://catalogue.dataspace.copernicus.eu/odata/v1/Products?%24filter=%28Collection%2FName+eq+%27SENTINEL-2%27%29+and+ContentDate%2FStart+gt+2022-05-03T00%3A00%3A00.000000Z+and+ContentDate%2FStart+lt+2022-05-03T10%3A00%3A00.000000Z&%24orderby=ContentDate%2FStart&%24top=100'

In [56]:
response_json = response.json()
values = pandas.DataFrame.from_dict(response_json["value"])
values.head(2)

,@odata.mediaContentType,Id,Name,ContentType,ContentLength,OriginDate,PublicationDate,ModificationDate,Online,EvictionDate,S3Path,Checksum,ContentDate,Footprint,GeoFootprint
0,application/octet-stream,b2ac733d-efa4-49e9-9edc-fb37e2a7f938,S2B_OPER_AUX_GNSSRD_POD__20220510T031816_V2022...,application/octet-stream,2650202,2022-05-10T03:32:10.993000Z,2023-10-25T13:45:42.482692Z,2023-11-29T14:03:28.169944Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/AUX/AUX_GNSSRD/2022/05/03/S...,"[{'Value': '5d7e3d41cedc5331584cdc6b92692b6c',...","{'Start': '2022-05-03T00:00:04.000000Z', 'End'...",None,None
1,application/octet-stream,a449f701-d29a-4ce9-b838-33061ee1d573,S2A_OPER_AUX_GNSSRD_POD__20220510T030151_V2022...,application/octet-stream,2281403,2022-05-10T03:32:11.048000Z,2023-10-25T13:45:41.052411Z,2023-11-29T14:03:27.695824Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/AUX/AUX_GNSSRD/2022/05/03/S...,"[{'Value': '56a1a8b66c90102c86511dfa991babc3',...","{'Start': '2022-05-03T00:00:07.000000Z', 'End'...",None,None


# Example Query - within location

In [86]:
start_date = datetime.datetime(year=2022, month=5, day=3)
time_range = datetime.timedelta(days=10)
start_string = start_date.strftime("%Y-%m-%dT%X.%fZ")
end_string = (start_date + time_range).strftime("%Y-%m-%dT%X.%fZ")
data_collection = "SENTINEL-2"

In [87]:
data_path = pathlib.Path.cwd() / ".." / "data"
boundary = geopandas.read_file(data_path / "vectors" / f"test.geojson")
boundary.to_wkt().iloc[0].geometry

'POLYGON ((171.149123 -46.732637, 171.207704 -44.937886, 169.380431 -44.894556, 169.262374 -46.686525, 171.149123 -46.732637))'

In [88]:
api_parameters = {
    "$filter": f"(Collection/Name eq '{data_collection}') "
               f"and OData.CSC.Intersects(area=geography'SRID=4326;{boundary.to_wkt().iloc[0].geometry}') "
               f"and ContentDate/Start gt {start_string} "
               f"and ContentDate/Start lt {end_string}",
    "$orderby": "ContentDate/Start",
    "$top": 20}
response = requests.get(data_url, params=api_parameters)
response.url

'https://catalogue.dataspace.copernicus.eu/odata/v1/Products?%24filter=%28Collection%2FName+eq+%27SENTINEL-2%27%29+and+OData.CSC.Intersects%28area%3Dgeography%27SRID%3D4326%3BPOLYGON+%28%28171.149123+-46.732637%2C+171.207704+-44.937886%2C+169.380431+-44.894556%2C+169.262374+-46.686525%2C+171.149123+-46.732637%29%29%27%29+and+ContentDate%2FStart+gt+2022-05-03T00%3A00%3A00.000000Z+and+ContentDate%2FStart+lt+2022-05-13T00%3A00%3A00.000000Z&%24orderby=ContentDate%2FStart&%24top=20'

In [89]:
response_json = response.json()
values = pandas.DataFrame.from_dict(response_json["value"])
values.head(2)

,@odata.mediaContentType,Id,Name,ContentType,ContentLength,OriginDate,PublicationDate,ModificationDate,Online,EvictionDate,S3Path,Checksum,ContentDate,Footprint,GeoFootprint
0,application/octet-stream,1cb6b808-7778-5ba4-980c-77709a495e32,S2B_MSIL2A_20220503T222539_N0400_R029_T59GML_2...,application/octet-stream,0,2022-05-04T05:54:34.427000Z,2022-05-04T06:00:21.338385Z,2022-05-04T06:00:21.338385Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A/2022/05/03/S2B_MSIL...,[],"{'Start': '2022-05-03T22:25:39.024000Z', 'End'...",geography'SRID=4326;POLYGON ((169.72822247963 ...,"{'type': 'Polygon', 'coordinates': [[[169.7282..."
1,application/octet-stream,27c46ab3-7424-5e7a-8988-c0c41f8a6c69,S2B_MSIL2A_20220503T222539_N0400_R029_T59GMJ_2...,application/octet-stream,0,2022-05-04T05:12:14.346000Z,2022-05-04T05:16:55.150653Z,2022-05-04T05:16:55.150653Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A/2022/05/03/S2B_MSIL...,[],"{'Start': '2022-05-03T22:25:39.024000Z', 'End'...",geography'SRID=4326;POLYGON ((169.70695108472 ...,"{'type': 'Polygon', 'coordinates': [[[169.7069..."


In [90]:
len(values)

20

# Example Query - cloudCover

In [77]:
start_date = datetime.datetime(year=2022, month=5, day=3)
time_range = datetime.timedelta(days=10)
start_string = start_date.strftime("%Y-%m-%dT%X.%fZ")
end_string = (start_date + time_range).strftime("%Y-%m-%dT%X.%fZ")
data_collection = "SENTINEL-2"
cloud_cover = 30

In [78]:
data_path = pathlib.Path.cwd() / ".." / "data"
boundary = geopandas.read_file(data_path / "vectors" / f"test.geojson")
boundary.to_wkt().iloc[0].geometry

'POLYGON ((171.149123 -46.732637, 171.207704 -44.937886, 169.380431 -44.894556, 169.262374 -46.686525, 171.149123 -46.732637))'

In [79]:
api_parameters = {
    "$filter": f"(Collection/Name eq '{data_collection}') "
               f"and OData.CSC.Intersects(area=geography'SRID=4326;{boundary.to_wkt().iloc[0].geometry}') "
               f"and Attributes/OData.CSC.DoubleAttribute/any(att:att/Name eq 'cloudCover' and att/OData.CSC.DoubleAttribute/Value le {cloud_cover}) "
               f"and ContentDate/Start gt {start_string} "
               f"and ContentDate/Start lt {end_string}",
    "$orderby": "ContentDate/Start",
    "$top": 20}
response = requests.get(data_url, params=api_parameters)
response.url

'https://catalogue.dataspace.copernicus.eu/odata/v1/Products?%24filter=%28Collection%2FName+eq+%27SENTINEL-2%27%29+and+OData.CSC.Intersects%28area%3Dgeography%27SRID%3D4326%3BPOLYGON+%28%28171.149123+-46.732637%2C+171.207704+-44.937886%2C+169.380431+-44.894556%2C+169.262374+-46.686525%2C+171.149123+-46.732637%29%29%27%29+and+Attributes%2FOData.CSC.DoubleAttribute%2Fany%28att%3Aatt%2FName+eq+%27cloudCover%27+and+att%2FOData.CSC.DoubleAttribute%2FValue+le+30%29+and+ContentDate%2FStart+gt+2022-05-03T00%3A00%3A00.000000Z+and+ContentDate%2FStart+lt+2022-05-13T00%3A00%3A00.000000Z&%24orderby=ContentDate%2FStart&%24top=20'

In [84]:
response_json = response.json()
values = pandas.DataFrame.from_dict(response_json["value"])
values.head(2)

,@odata.mediaContentType,Id,Name,ContentType,ContentLength,OriginDate,PublicationDate,ModificationDate,Online,EvictionDate,S3Path,Checksum,ContentDate,Footprint,GeoFootprint
0,application/octet-stream,1cb6b808-7778-5ba4-980c-77709a495e32,S2B_MSIL2A_20220503T222539_N0400_R029_T59GML_2...,application/octet-stream,0,2022-05-04T05:54:34.427000Z,2022-05-04T06:00:21.338385Z,2022-05-04T06:00:21.338385Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A/2022/05/03/S2B_MSIL...,[],"{'Start': '2022-05-03T22:25:39.024000Z', 'End'...",geography'SRID=4326;POLYGON ((169.72822247963 ...,"{'type': 'Polygon', 'coordinates': [[[169.7282..."
1,application/octet-stream,27c46ab3-7424-5e7a-8988-c0c41f8a6c69,S2B_MSIL2A_20220503T222539_N0400_R029_T59GMJ_2...,application/octet-stream,0,2022-05-04T05:12:14.346000Z,2022-05-04T05:16:55.150653Z,2022-05-04T05:16:55.150653Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A/2022/05/03/S2B_MSIL...,[],"{'Start': '2022-05-03T22:25:39.024000Z', 'End'...",geography'SRID=4326;POLYGON ((169.70695108472 ...,"{'type': 'Polygon', 'coordinates': [[[169.7069..."


In [85]:
len(values)

15

## Processing API
* Documentation at [Processing API](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Process.html)
* NVDI 